In [3]:
import gtsam
import numpy as np
from operator import itemgetter

In [2]:
#### FILTER OBJECT
class GraphTracker():
    def __init__(self):
        
        # Member variables
        self.init = False
        self.detections = []
        self.tracks = []
        self.graph = gtsam.NonlinearFactorGraph()
        self.trk_delete_thresh = 0.25
        self.trk_delete_age = 10
        self.trk_create_thresh = 0.75

    # Member functions
    def handle_detections(self,det_array_msg):
        self.detections = det_array_msg.boxes
        
        # TODO - Compute detection-track correspondences
        
        # TODO - Assign detections to track variables
        
        # TODO - create tracks as needed
        
        # Delete tracks as needed
        self.delete_tracks()
        
    def delete_tracks(self):
        self.tracks = [track for track in self.tracks if track.prob < self.trk_delete_thresh]
        self.tracks = [track for track in self.tracks if track.age > self.trk_delete_age]
        
    # birth
    # compute assignment
    # add factors
    # optimize

In [5]:
#### DETECTION OBJECT
class Detection():
    def __init__(self, det_msg):
        self.px = det_msg.pose.position.x
        self.py = det_msg.pose.position.y
        self.pz = det_msg.pose.position.z
        self.age = 0
        self.prob = det_msg.value
        self.class = det_msg.label
        # Can be empty/null/contain partial info
        # class conf % - float
        # ??? BB dimensions

#### TRACK OBJECT
class Track():
    def __init__(self, det):
        self.px = det.px
        self.py = det.py
        self.pz = det.pz
        self.vx = 0
        self.vy = 0
        self.vz = 0
        self.prob = 
        self.age = det.age
        # heading in world frame
        # self.class = categorical distribution!
        # ??? class conf %

In [ ]:
#### HELPER FUNCTIONS
# Similarity metric (detection, track)

# Form similarity matrix (detections, tracks) -> return similarity matrix
# - call similarity metric

# Hungarian/Munkres algo (similarity matrix) -> return assignments (T_match, D_match, T_unmatch (missed/occ), D_unmatch (false, new obj, extra detection)


In [4]:
import numpy as np
from scipy.optimize import linear_sum_assignment

# Create a cost matrix
cost_matrix = np.array([
    [4, 1, 3],
    [2, 0, 5],
    [3, 2, 2],
    [0, 0, 0]
])

# Solve the assignment problem
row_indices, col_indices = linear_sum_assignment(cost_matrix)

# Extract the optimal assignment
assignment = [(row, col) for row, col in zip(row_indices, col_indices)]

print("Optimal Assignment:")
for row, col in assignment:
    print(f"Resource {row} assigned to Task {col}")


Optimal Assignment:
Resource 1 assigned to Task 1
Resource 2 assigned to Task 2
Resource 3 assigned to Task 0


In [5]:
def cost(det,track):
    return np.abs(det-track)

def compute_cost_matrix(detects, tracks):
    cost_matrix = np.zeros((len(detects),len(tracks)))
    for ii,det in enumerate(detects):
        for jj,trk in enumerate(tracks):
            cost_matrix[ii,jj] = cost(det,trk)
    return cost_matrix  

tracks = [1, 2, 3, 9, 9]
# tracks = [1, 2, 3]
det = [3.1, 3.01, 0.9, 4.4, 2.1]

cost_matrix = compute_cost_matrix(det,tracks)

# Solve the assignment problem
row_indices, col_indices = linear_sum_assignment(cost_matrix)

# Extract the optimal assignment
assignment = [(row, col) for row, col in zip(row_indices, col_indices)]

In [7]:
row_indices, col_indices = linear_sum_assignment(cost_matrix)
row_indices, col_indices = list(row_indices), list(col_indices)
print(row_indices)
print(col_indices)


[0, 1, 2, 3, 4]
[4, 2, 0, 3, 1]


In [16]:
row_indices, col_indices = linear_sum_assignment(cost_matrix)
row_indices, col_indices = list(row_indices), list(col_indices)

print(cost_matrix)
print()
print("Assign this detection")
print(row_indices)
print("To this track")
print(col_indices)
# print("Assign this detection")
# print(row_indices)
# print("To this track")
# print(col_indices)
# print(assignment)

trk_thresh = 4.
del_indices = []

assert(len(row_indices) == len(col_indices))
# for ii in range(len(row_indices)):
#     det_idx = row_indices[ii]
#     trk_idx = col_indices[ii]
    
#     print("Cost of assigning det " + str(det[det_idx]) + " to track " + str(tracks[trk_idx]))
#     print("is " + str(cost_matrix[det_idx,trk_idx]))

#     if cost_matrix[det_idx,trk_idx] > trk_thresh:
#         print('too costly')
#         del_indices.append(ii)
        
        
# print()
# print("Delete these match indices:")
# print(del_indices)
# while del_indices:
#     idx = del_indices.pop()
#     print(idx)
#     del row_indices[idx]
#     del col_indices[idx]

ii = len(row_indices)
while ii:
    idx = ii-1
    if cost_matrix[row_indices[idx],col_indices[idx]] > trk_thresh:
        print(idx)
        print('too high')
        del row_indices[idx], col_indices[idx]       
    ii -=1
       
print()
print("Assign this detection")
print(det)
print(row_indices)
print("To this track")
print(tracks)
print(col_indices)


    

[[2.1  1.1  0.1  5.9  5.9 ]
 [2.01 1.01 0.01 5.99 5.99]
 [0.1  1.1  2.1  8.1  8.1 ]
 [3.4  2.4  1.4  4.6  4.6 ]
 [1.1  0.1  0.9  6.9  6.9 ]]

Assign this detection
[0, 1, 2, 3, 4]
To this track
[4, 2, 0, 3, 1]
3
too high
0
too high

Assign this detection
[3.1, 3.01, 0.9, 4.4, 2.1]
[1, 2, 4]
To this track
[1, 2, 3, 9, 9]
[2, 0, 1]


In [206]:
print(det)
print(list(row_indices))
print()

match_det = [det[i] for i in list(row_indices)]
match_trk = [tracks[i] for i in list(col_indices)]
print(match_det)
print(match_trk)
print()

unmatched_det = [det[i] for i in range(len(det)) if i not in list(row_indices)]
unmatched_trk = [tracks[i] for i in range(len(tracks)) if i not in list(col_indices)]
print(unmatched_det)
print(unmatched_trk)

new_det = det.copy()
new_trk = []


while new_det:
    last_idx = len(new_det)-1
    print(last_idx)
    print(last_idx in row_indices)
    new_trk.append(new_det.pop()) if (last_idx in row_indices) else new_det.pop()
    
    
print(new_det)
print(new_trk)


[3.1, 3.01, 0.9, 4.4, 2.1]
[1, 2, 4]

[3.01, 0.9, 2.1]
[3, 1, 2]

[3.1, 4.4]
[]
4
True
3
False
2
True
1
True
0
False
[]
[2.1, 0.9, 3.01]


In [173]:
print(list(row_indices))

print(det)
matched_det = [det[i] for i in row_indices]
# matched_det = det.pop(row_indices)
print(matched_det)

del [det[i] for i in list(row_indices)]
unmatched_det = det
print(unmatched_det)

SyntaxError: cannot delete list comprehension (1955088690.py, line 8)

In [21]:
import numpy as np
my_arr = np.array([[0.],[0.],[0.]])
my_arr2 = np.array([[1.],[2.],[3.]])


In [24]:
np.linalg.norm(my_arr - my_arr2)

3.7416573867739413

# Kalman filter testing

In [56]:
# Initialization
pos = np.array([[1],[2],[3]])
vel = np.array([[3],[2],[1]])

x0 = np.vstack((pos,vel))
P0 = np.diag([0.5, 0.5, 0.5, 0.5, 0.5, 0.5])

kf = gtsam.KalmanFilter(6)
state = kf.init(x0, P0)
print(state.mean()[3:6])

[3. 2. 1.]


In [6]:
dir(kf)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'init',
 'predict',
 'predict2',
 'predictQ',
 'print',
 'step',
 'update',
 'updateQ']

In [50]:
# prediction
dt = .03

F = np.array([[1., 0, 0, dt, 0, 0],
               [0, 1., 0, 0, dt, 0],
               [0, 0, 1., 0, 0, dt],
               [0, 0, 0, 1., 0, 0],
               [0, 0, 0, 0, 1., 0],
               [0, 0, 0, 0, 0, 1.]])

Q = gtsam.noiseModel.Diagonal.Sigmas([.1,.1,.1,.1,.1,.1])
print(Q)

kf.predict(state,F,np.zeros((6,6)),np.zeros((6,1)),Q)
print(state.mean()[3:6])

isotropic dim=6 sigma=0.1

[0. 0. 0.]


In [46]:
obs_model = np.array([[1, 0, 0, 0, 0, 0],
                     [0, 1, 0, 0, 0, 0],
                     [0, 0, 1, 0, 0, 0]])
print(obs_model)
meas = np.array([[2],[3],[4]])
print(meas)
meas_noise = gtsam.noiseModel.Diagonal.Sigmas([.1,.1,.1])

kf.update(state, obs_model, meas, meas_noise)

[[1 0 0 0 0 0]
 [0 1 0 0 0 0]
 [0 0 1 0 0 0]]
[[2]
 [3]
 [4]]


GaussianDensity: density on [0] 
mean: [1.98039216; 2.98039216; 3.98039216; 0; 0; 0];
covariance: [
	0.00980392157, 0, 0, 0, 0, 0;
	0, 0.00980392157, 0, 0, 0, 0;
	0, 0, 0.00980392157, 0, 0, 0;
	0, 0, 0, 0.5, 0, 0;
	0, 0, 0, 0, 0.5, 0;
	0, 0, 0, 0, 0, 0.5
]
Noise model: unit (6) 

In [47]:
dir(gtsam)

['AHRSFactor',
 'AcceleratingScenario',
 'BatchFixedLagSmoother',
 'BearingFactor2D',
 'BearingFactor3D',
 'BearingFactorPose2',
 'BearingRange2D',
 'BearingRange3D',
 'BearingRangeFactor2D',
 'BearingRangeFactor3D',
 'BearingRangeFactorPose2',
 'BearingRangeFactorPose3',
 'BearingRangePose2',
 'BearingRangePose3',
 'BetweenFactorConstantBias',
 'BetweenFactorDouble',
 'BetweenFactorPoint2',
 'BetweenFactorPoint3',
 'BetweenFactorPose2',
 'BetweenFactorPose2s',
 'BetweenFactorPose3',
 'BetweenFactorPose3s',
 'BetweenFactorRot2',
 'BetweenFactorRot3',
 'BetweenFactorSO3',
 'BetweenFactorSO4',
 'BetweenFactorVector',
 'BinaryMeasurementPoint3',
 'BinaryMeasurementRot3',
 'BinaryMeasurementUnit3',
 'BinaryMeasurementsPoint3',
 'BinaryMeasurementsRot3',
 'BinaryMeasurementsUnit3',
 'BlockJacobiPreconditionerParameters',
 'Cal3Bundler',
 'Cal3DS2',
 'Cal3DS2_Base',
 'Cal3Fisheye',
 'Cal3Unified',
 'Cal3_S2',
 'Cal3_S2Stereo',
 'CalibratedCamera',
 'CameraSetCal3Bundler',
 'CameraSetCal3DS2'